# Entrenamiento de un Clasificador Naive Bayes

En esta hoja de trabajo veremos cómo entrenar un clasificador Naive Bayes y evaluar su rendimiento. Empezaremos importando las librerías y funciones que utilizaremos más adelante.


In [1]:
import pandas as pd
import csv
import numpy as np
np.random.seed(357823)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

# Lectura de datos y creación de los atributos y el objetivo

Vamos a empezar por importar el dataset con todos los atributos que calculamos para la realización de este trabajo. Luego, separamos el dataset en atributos por los cuales Naive Bayes se va a entrenar. Finalmente, elegiremos el objetivo, que en este caso es un objetivo categórico binario.

In [2]:
# Lectura del csv para la tabla con todas las métricas
nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")

# Lectura del csv para la tabla con todos los atributos sin los de agrupamiento
nodesWithoutClustering = pd.read_csv("../tablas/tableWithoutClustering.csv")

# Lectura del csv para la tabla con todos los atributos sin los de comunidades
nodesWithoutCommunity = pd.read_csv("../tablas/tableWithoutCommunity.csv")

#Tablas con todos los atributos sin los de nucleos
nodesWithoutKernel = pd.read_csv("../tablas/tableWithoutKernel.csv")


# Tabla con todos los atributos

In [3]:
nodes.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


# Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [4]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodes.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [5]:
# Objetivo
objetivo = nodes['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [6]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()


///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [7]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,124.0,70.0,0.0,1.0,106.0,0.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,0.0,18.0,0.0,3.0,63.0,0.0
4,4,0.969442,0.0,186.0,0.0,0.0,0.0,0.0,7.0,21.0,0.0


# Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [8]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [9]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [10]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [11]:
búsqueda_en_rejilla.best_score_

0.7260724362721616

# Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.


In [12]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [13]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [14]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 1., 0.])

In [15]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               4749                843
Verdadero Negativo                850               1098


In [16]:
recall_score(objetivo_prueba, predicciones)

0.5636550308008214

# Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [17]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [18]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [19]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.03845811, 0.036901  , 0.04152656, 0.03390908, 0.03423262,
        0.03692961, 0.03445101, 0.03606153, 0.03535771, 0.03681636]),
 'score_time': array([0.0059557 , 0.00698185, 0.00598383, 0.00598407, 0.00501513,
        0.00495791, 0.00553775, 0.00700808, 0.0069809 , 0.00797248]),
 'test_score': array([0.94552929, 0.67659138, 0.68172485, 0.65195072, 0.68069815,
        0.66016427, 0.70020534, 0.66427105, 0.65297741, 0.94661191])}

In [20]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.03989196, 0.03760147, 0.03390956, 0.03251553, 0.03384256,
        0.03404188, 0.03346944, 0.03560829, 0.03437781, 0.05037141]),
 'score_time': array([0.00598454, 0.00598431, 0.00498605, 0.00600934, 0.00595546,
        0.00501513, 0.00598454, 0.00598359, 0.0074904 , 0.01396275]),
 'test_score': array([0.90030832, 0.62422998, 0.61601643, 0.5862423 , 0.62320329,
        0.61396304, 0.63244353, 0.60985626, 0.59753593, 0.89322382])}

In [21]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.7260724362721616

In [22]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.6697022903824198

# Análisis del resultado

Al analizar nuestros resultados, observamos que el mejor puntaje se logra al establecer un valor de alpha igual a 1. Esto nos proporciona una confianza del 72.61%. Dado que en nuestros datos el 75% corresponden a desarrolladores web y el 25% a desarrolladores de IA, al examinar la matriz de confusión, concluimos que el árbol de decisión está mejor entrenado que un clasificador aleatorio. Aunque hay una cantidad significativa de falsos positivos y falsos negativos, esto podría ser atribuible a la escasez de ejemplos de desarrolladores de IA en la partición de los datos, lo que sugiere que el modelo tiene un rendimiento aceptable dadas las condiciones de partición.